In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-03-24 20:20:32.577145: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 20:20:32.604757: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 20:20:32.604782: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 20:20:32.604808: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 20:20:32.610620: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-24 20:20:34.320437: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 20:20:34.347150: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 20:20:34.347375: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
train_data_path = "BIRDS1_split/train"
test_data_path = "BIRDS1_split/test"
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
bird_classes = sorted(os.listdir(train_data_path))

In [3]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_data_path,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)


Found 25744 files belonging to 167 classes.
Using 18021 files for training.


2024-03-24 20:20:34.945340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 20:20:34.945621: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 20:20:34.945795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 25744 files belonging to 167 classes.
Using 7723 files for validation.
Found 11545 files belonging to 167 classes.


In [4]:
preprocess_input = tf.keras.applications.resnet.preprocess_input
train_dataset = train_dataset.map(lambda x, y: (preprocess_input(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_dataset = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
base_model = tf.keras.applications.ResNet152(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
)
base_model.trainable = False

for layer in base_model.layers:
    print(layer.name)
len(base_model.layers)

input_1
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn
conv3_

515

In [6]:
from keras.src import regularizers

inputs = layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name="input_layer")

x = base_model(inputs, training=False)
x = layers.GlobalAvgPool2D()(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = layers.Dense(len(bird_classes), activation='softmax', dtype=tf.float32, kernel_regularizer=regularizers.l2(0.005))(x)

model1 = tf.keras.Model(inputs, outputs)

In [7]:
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                             

In [8]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.5, patience=3) 

In [9]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])

checkpointer = ModelCheckpoint(filepath='Best_models/resnet/', verbose=1, save_best_only=True)

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=10,
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 1/10


2024-03-24 20:20:44.183851: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-03-24 20:20:44.305449: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-24 20:20:45.414868: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x75e3e80b91a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-24 20:20:45.414885: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-24 20:20:45.419682: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-24 20:20:45.489308: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


564/564 [==============================] - ETA: 0s - loss: 3.5224 - accuracy: 0.2496
Epoch 1: val_loss improved from inf to 1.50636, saving model to Best_models/resnet/
564/564 [==============================] - 91s 151ms/step - loss: 3.5224 - accuracy: 0.2496 - val_loss: 1.5064 - val_accuracy: 0.6844 - lr: 0.0010
Epoch 2/10
563/564 [============================>.] - ETA: 0s - loss: 1.8509 - accuracy: 0.5467
Epoch 2: val_loss improved from 1.50636 to 1.06782, saving model to Best_models/resnet/
564/564 [==============================] - 84s 148ms/step - loss: 1.8510 - accuracy: 0.5467 - val_loss: 1.0678 - val_accuracy: 0.7603 - lr: 0.0010
Epoch 3/10
563/564 [============================>.] - ETA: 0s - loss: 1.4732 - accuracy: 0.6314
Epoch 3: val_loss improved from 1.06782 to 0.86525, saving model to Best_models/resnet/
564/564 [==============================] - 78s 138ms/step - loss: 1.4732 - accuracy: 0.6314 - val_loss: 0.8652 - val_accuracy: 0.8021 - lr: 0.0010
Epoch 4/10
563/564 [==

In [10]:
for layer in base_model.layers[-100:]:
    layer.trainable = True


In [11]:
for layer in base_model.layers:
    print(layer.name, layer.trainable)

input_1 False
conv1_pad False
conv1_conv False
conv1_bn False
conv1_relu False
pool1_pad False
pool1_pool False
conv2_block1_1_conv False
conv2_block1_1_bn False
conv2_block1_1_relu False
conv2_block1_2_conv False
conv2_block1_2_bn False
conv2_block1_2_relu False
conv2_block1_0_conv False
conv2_block1_3_conv False
conv2_block1_0_bn False
conv2_block1_3_bn False
conv2_block1_add False
conv2_block1_out False
conv2_block2_1_conv False
conv2_block2_1_bn False
conv2_block2_1_relu False
conv2_block2_2_conv False
conv2_block2_2_bn False
conv2_block2_2_relu False
conv2_block2_3_conv False
conv2_block2_3_bn False
conv2_block2_add False
conv2_block2_out False
conv2_block3_1_conv False
conv2_block3_1_bn False
conv2_block3_1_relu False
conv2_block3_2_conv False
conv2_block3_2_bn False
conv2_block3_2_relu False
conv2_block3_3_conv False
conv2_block3_3_bn False
conv2_block3_add False
conv2_block3_out False
conv3_block1_1_conv False
conv3_block1_1_bn False
conv3_block1_1_relu False
conv3_block1_2_con

In [12]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(),
                           metrics=['accuracy'])
start_epoch = 10

model1.fit(train_dataset,
                                        validation_data=validation_dataset,
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 11/25
563/564 [============================>.] - ETA: 0s - loss: 0.8130 - accuracy: 0.7943
Epoch 11: val_loss improved from 0.52178 to 0.49966, saving model to Best_models/resnet/
564/564 [==============================] - 74s 125ms/step - loss: 0.8129 - accuracy: 0.7944 - val_loss: 0.4997 - val_accuracy: 0.8940 - lr: 0.0010
Epoch 12/25
563/564 [============================>.] - ETA: 0s - loss: 0.7854 - accuracy: 0.8005
Epoch 12: val_loss did not improve from 0.49966
564/564 [==============================] - 51s 90ms/step - loss: 0.7853 - accuracy: 0.8006 - val_loss: 0.5096 - val_accuracy: 0.8845 - lr: 0.0010
Epoch 13/25
563/564 [============================>.] - ETA: 0s - loss: 0.7545 - accuracy: 0.8107
Epoch 13: val_loss improved from 0.49966 to 0.49183, saving model to Best_models/resnet/
564/564 [==============================] - 70s 124ms/step - loss: 0.7546 - accuracy: 0.8107 - val_loss: 0.4918 - val_accuracy: 0.8950 - lr: 0.0010
Epoch 14/25
563/564 [======================

In [13]:
len(base_model.layers)

515

In [14]:

for layer in base_model.layers[-250:]:
    layer.trainable = True


In [16]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.0003),
                           metrics=['accuracy'])
start_epoch = 25

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 26/40
563/564 [============================>.] - ETA: 0s - loss: 0.4546 - accuracy: 0.8763
Epoch 26: val_loss improved from 0.35077 to 0.32980, saving model to Best_models/resnet/
564/564 [==============================] - 82s 138ms/step - loss: 0.4545 - accuracy: 0.8763 - val_loss: 0.3298 - val_accuracy: 0.9066 - lr: 3.0000e-04
Epoch 27/40
563/564 [============================>.] - ETA: 0s - loss: 0.4291 - accuracy: 0.8885
Epoch 27: val_loss did not improve from 0.32980
564/564 [==============================] - 54s 95ms/step - loss: 0.4291 - accuracy: 0.8885 - val_loss: 0.3384 - val_accuracy: 0.9033 - lr: 3.0000e-04
Epoch 28/40
563/564 [============================>.] - ETA: 0s - loss: 0.4181 - accuracy: 0.8903
Epoch 28: val_loss improved from 0.32980 to 0.32474, saving model to Best_models/resnet/
564/564 [==============================] - 74s 132ms/step - loss: 0.4183 - accuracy: 0.8902 - val_loss: 0.3247 - val_accuracy: 0.9114 - lr: 3.0000e-04
Epoch 29/40
563/564 [==========

KeyboardInterrupt: 

In [17]:
for layer in base_model.layers[-400:]:
    layer.trainable = True


In [18]:
model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.0001),
                           metrics=['accuracy'])
start_epoch = 37

model1.fit(train_dataset,
                                       
                                        validation_data=validation_dataset,
                                       
                                        epochs=start_epoch+15,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 38/52
563/564 [============================>.] - ETA: 0s - loss: 0.3435 - accuracy: 0.9071
Epoch 38: val_loss improved from 0.28811 to 0.27944, saving model to Best_models/resnet/
564/564 [==============================] - 83s 140ms/step - loss: 0.3435 - accuracy: 0.9071 - val_loss: 0.2794 - val_accuracy: 0.9214 - lr: 1.0000e-04
Epoch 39/52
563/564 [============================>.] - ETA: 0s - loss: 0.3435 - accuracy: 0.9069
Epoch 39: val_loss improved from 0.27944 to 0.27544, saving model to Best_models/resnet/
564/564 [==============================] - 79s 139ms/step - loss: 0.3436 - accuracy: 0.9069 - val_loss: 0.2754 - val_accuracy: 0.9193 - lr: 1.0000e-04
Epoch 40/52
563/564 [============================>.] - ETA: 0s - loss: 0.3394 - accuracy: 0.9097
Epoch 40: val_loss improved from 0.27544 to 0.27403, saving model to Best_models/resnet/
564/564 [==============================] - 79s 140ms/step - loss: 0.3393 - accuracy: 0.9098 - val_loss: 0.2740 - val_accuracy: 0.9283 - lr: 

KeyboardInterrupt: 

In [22]:
for layer in base_model.layers[-515:]:
    layer.trainable = True

model1.compile(loss='categorical_crossentropy',
                           optimizer=Adam(0.000006),
                           metrics=['accuracy'])
start_epoch = 78

model1.fit(train_dataset,
                                        
                                        validation_data=validation_dataset,
                                        
                                        epochs=start_epoch+30,
                                        initial_epoch=start_epoch, 
                                        verbose=1,
                                        callbacks=[checkpointer, lr_scheduler]) 

Epoch 79/108
563/564 [============================>.] - ETA: 0s - loss: 0.2951 - accuracy: 0.9202
Epoch 79: val_loss improved from 0.25861 to 0.25718, saving model to Best_models/resnet/
564/564 [==============================] - 81s 137ms/step - loss: 0.2952 - accuracy: 0.9202 - val_loss: 0.2572 - val_accuracy: 0.9548 - lr: 6.0000e-06
Epoch 80/108
563/564 [============================>.] - ETA: 0s - loss: 0.2949 - accuracy: 0.9223
Epoch 80: val_loss did not improve from 0.25718
564/564 [==============================] - 53s 93ms/step - loss: 0.2949 - accuracy: 0.9223 - val_loss: 0.2576 - val_accuracy: 0.9527 - lr: 6.0000e-06
Epoch 81/108
563/564 [============================>.] - ETA: 0s - loss: 0.2923 - accuracy: 0.9228
Epoch 81: val_loss improved from 0.25718 to 0.25619, saving model to Best_models/resnet/
564/564 [==============================] - 74s 131ms/step - loss: 0.2923 - accuracy: 0.9228 - val_loss: 0.2562 - val_accuracy: 0.9551 - lr: 6.0000e-06
Epoch 82/108
563/564 [======

KeyboardInterrupt: 

In [24]:
loaded = tf.keras.models.load_model("Best_models/resnet")
loaded.evaluate(test_dataset)

361/361 [==============================] - 27s 71ms/step - loss: 0.3528 - accuracy: 0.9023


[0.3527677059173584, 0.9022953510284424]

In [25]:
y_labels_224 = []
for images, labels in test_dataset.unbatch():
    y_labels_224.append(labels.numpy().argmax())


In [26]:
def calculate_accuracy_top3(row):
    if row['True Label'] in row['top 3 cols']:
        return 1
    else:
        return 0

In [27]:
import pandas as pd


preds_eff_b7 = pd.DataFrame(loaded.predict(test_dataset))
top3_labels = utils.add_top_three_row_values(preds_eff_b7, y_labels_224)
top3_labels['Accuracy_Top3'] = top3_labels.apply(calculate_accuracy_top3, axis=1)
overall_accuracy_top3 = top3_labels['Accuracy_Top3'].mean()
overall_accuracy_top3

361/361 [==============================] - 26s 68ms/step


0.9871805976613253

In [1]:
import tensorflow as tf

loaded_resnet = tf.keras.models.load_model("Best_models/resnet_90.22")
loaded_resnet.summary()

2024-03-24 22:09:03.446723: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-24 22:09:03.693485: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 22:09:03.693524: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 22:09:03.694752: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 22:09:03.803456: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-03-24 22:09:05.466603: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 22:09:05.571781: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-24 22:09:05.572080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                16448     
                                                             